# Importando bibliotecas

In [3]:

import numpy as np
import pandas as pd
import random as rd


No GPU was detected. LSTMs and CNNs can be very slow without a GPU.


# Lê o arquivo

O arquivo contém dados de endereços junto com a geolocalização (latitude e longitude)

O problema é que o arquivo é estruturado, devemos preparar os dados para serem aprendidos por uma IA.

In [1]:

df = pd.read_csv("reduced2.csv", index_col = 0)
df.info()
df.head()

df.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 63274 entries, 0 to 63273
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0.1  63274 non-null  int64  
 1   LON           63274 non-null  float64
 2   LAT           63274 non-null  float64
 3   NUMBER        62304 non-null  object 
 4   STREET        63274 non-null  object 
 5   UNIT          25556 non-null  object 
 6   CITY          63241 non-null  object 
 7   DISTRICT      0 non-null      float64
 8   REGION        62815 non-null  object 
 9   POSTCODE      63274 non-null  object 
 10  ID            0 non-null      float64
 11  HASH          63274 non-null  object 
dtypes: float64(4), int64(1), object(7)
memory usage: 6.3+ MB


,Unnamed: 0.1,LON,LAT,DISTRICT,ID
count,63274.000000,63274.000000,63274.000000,0.0,0.0
mean,318012.685005,-45.877759,-17.390369,NaN,NaN
std,183068.666365,6.015264,8.067416,NaN,NaN
min,6.000000,-73.055613,-33.683350,NaN,NaN
25%,159718.000000,-49.332333,-23.395961,NaN,NaN
50%,318041.500000,-46.395296,-20.099695,NaN,NaN
75%,476779.500000,-41.856481,-10.325681,NaN,NaN
max,633778.000000,-34.807311,4.490910,NaN,NaN


# Função responsável por aleatorizar o nome do endereço

Ele prepara o dado para ser lido pela inteligencia artificial, alterando-se de várias maneiras a forma como o endereço é escrito.

In [ ]:
def random_address_generator(num, rua, ext, city, dist, reg, cep):

    long = rd.randint(0,1)
    if long:
        number_pos = [""]
    else:
        number_pos = ["nº ","número ",""]

    try:
        if num:
            int(num)
        else:
            number_pos = [""]
    except ValueError:
        number_pos = [""]

    num_pos = rd.choice(number_pos)
    has_ext = rd.randint(0,1)
    
    take_out_coma_prob = 0.2

    comas = rd.choices([",",''], weights = [1, take_out_coma_prob], k = 4) 
    
    if long:
        spaces = [rd.choice([" ",'']) for i in range(4)]
        bars = [rd.choice(["/",'-']) for i in range(2)]
        address = f"{rua}{comas[0]} {num_pos}{num}{comas[1]} {city}{spaces[0]}{bars[0]}{spaces[1]}{reg}{spaces[2]}{bars[1]}{spaces[3]}CEP {cep}"    # RUA EXPEDICIONÁRIO BRASILIO PINTO DE ALMEIDA, 42, Guararema-SP - CEP 08900-000
    else:
        more_things = rd.choice(["bairro ", ""])
        if has_ext and ext:
            address = f"{rua}{comas[0]} {num_pos}{num}{comas[1]} {ext}{comas[2]} {more_things}{city}"                    # Rua Admeleto Gasparini, número 1949, bairro Itaoca
        else:
            if dist != 0:
                address = f"{rua}{comas[0]} {num_pos}{num}{comas[1]} {more_things}{dist}"     # Rua Angico, fundos do nº 39, Taquaral
            else:
                address = f"{rua}{comas[0]} {num_pos}{num}{comas[1]} {more_things}{city}"     # Rua Angico, fundos do nº 39, Guararema
    return address

# Executa e escreve o arquivo .txt

In [65]:
nums = np.array(df['NUMBER'])
ruas = np.array(df['STREET'])
exts = np.array(df['UNIT'].fillna(0))
cids = np.array(df['CITY'])
dists = np.array(df['DISTRICT'].fillna(0))
regs = np.array(df['REGION'])
ceps = np.array(df['POSTCODE'])

addresses = [random_address_generator(nums[i], ruas[i], exts[i], cids[i], dists[i], regs[i], ceps[i]) for i in range(len(nums))]

print(addresses[:10])

with open('address_syntetic.txt', 'w') as f:
        for item in addresses:
            try:
                f.writelines(["%s\n" % item])
            except:
                pass
        

['RUA  GERALDO BARBOSA nº 75 FRENTE, Acrelândia', 'RUA  DOS PIONEIROS 2044, Acrelândia/AC / CEP 69945-000', 'RUA  JOSE ALVES DOS SANTOS, 146, Acrelândia/AC/ CEP 69945-000', 'RUA  PEDRO RODRIGUES, 50, Acrelândia - AC -CEP 69945-000', 'RAMAL  GRANADA, 129 CCE, Acrelândia/AC - CEP 69945-000', 'RAMAL  NOVO ENCANTO, 51 CCE, Acrelândia/ AC -CEP 69945-000', 'RAMAL  DO KM 20, 52 CCE, Acrelândia/AC/ CEP 69945-000', 'RUA  RAIMUNDO CHAAR, 462, Assis Brasil /AC- CEP 69935-000', 'AVENIDA PRESIDENTE GETULIO VARGAS SN, Assis Brasil/ AC /CEP 69935-000', 'RAMAL SAO JOSE, 1952, bairro Assis Brasil']
